In [1]:
import torch
from transformers import BertModel, GPT2LMHeadModel, BertTokenizer, GPT2Tokenizer

C:\Users\josep\anaconda3\envs\nlp-39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import torch
from transformers import BertModel, T5ForConditionalGeneration, BertTokenizer, T5Tokenizer
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# Cargar modelos y tokenizadores
bert_model_name = 'bert-base-multilingual-cased'
t5_model_name = 't5-base'
tokenizer_bert = BertTokenizer.from_pretrained(bert_model_name)
tokenizer_t5 = T5Tokenizer.from_pretrained(t5_model_name)
encoder = BertModel.from_pretrained(bert_model_name)
decoder = T5ForConditionalGeneration.from_pretrained(t5_model_name)

Downloading: 100%|██████████████████████████████████████████████████████████████████| 792k/792k [00:00<00:00, 1.23MB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████| 1.21k/1.21k [00:00<00:00, 594kB/s]
C:\Users\josep\anaconda3\envs\nlp-39\lib\site-packages\transformers\models\t5\tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings

In [12]:
# Cargar el DataFrame
data = pd.DataFrame({ "texto" : ["""Generative Pre-trained Transformer 3 (Transformador preentrenado generativo) , conocida por sus siglas (GPT-3), es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es la tercera generación de los modelos de predicción de lenguaje perteneciente a la serie GPT, creados por OpenAI, un laboratorio de investigación de inteligencia artificial con sede en San Francisco.1​ La versión completa de GPT-3 tiene una capacidad de 175.000 millones de parámetros de aprendizaje automatizado, lo cual supera la magnitud de su predecesor, GPT-2. GPT-3 fue introducido en mayo de 2020 y, hasta julio de 2020, se encontraba en fase beta.2​ Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas".3​ Previo a la liberación de GPT-3, el modelo de lenguaje más grande era Turing NLG desarrollado por Microsoft, presentado en febrero de 2020, con una capacidad diez veces menor que el de GPT-3."""]*100,
                       "resumen_texto" : ["""GPT-3 es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas"""]*100})
data

,texto,resumen_texto
0,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
1,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
2,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
3,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
4,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
...,...,...
95,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
96,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
97,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
98,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...


In [13]:



# Definir una clase personalizada de conjunto de datos
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer_bert, tokenizer_t5, max_length):
        self.data = data
        self.tokenizer_bert = tokenizer_bert
        self.tokenizer_t5 = tokenizer_t5
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        text = row["texto"]
        summary = row["resumen_texto"]

        # Codificar el texto y el resumen
        text_encoded = self.tokenizer_bert.encode_plus(text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        summary_encoded = self.tokenizer_t5.encode_plus(summary, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            "input_ids": text_encoded["input_ids"].squeeze(),
            "attention_mask": text_encoded["attention_mask"].squeeze(),
            "decoder_input_ids": summary_encoded["input_ids"].squeeze(),
            "decoder_attention_mask": summary_encoded["attention_mask"].squeeze(),
            "labels": summary_encoded["input_ids"].squeeze()
        }





In [14]:
# Crear el conjunto de datos y el DataLoader
max_length = 512
dataset = CustomDataset(data, tokenizer_bert, tokenizer_t5, max_length)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Entrenamiento del modelo
num_epochs = 3
optimizer = torch.optim.Adam(decoder.parameters(), lr=1e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder.to(device)
decoder.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [16]:
ls

 Volume in drive C is NVMe
 Volume Serial Number is FEEE-7323

 Directory of C:\Users\josep\Documents\Python Tesis

05/09/2023  10:31 PM    <DIR>          .
05/09/2023  07:44 AM    <DIR>          ..
05/09/2023  10:16 PM    <DIR>          .ipynb_checkpoints
05/09/2023  10:31 PM            10,713 Bert2GPT.ipynb
11/14/2022  04:42 PM        80,984,626 en-es-tsv.tsv
11/15/2022  11:20 AM        12,189,320 train_data_sample_15perc.csv
11/17/2022  02:47 PM           818,370 Untitled.ipynb
12/15/2022  09:22 PM            15,113 Untitled1.ipynb
05/09/2023  07:47 AM            60,915 Untitled2.ipynb
               6 File(s)     94,079,057 bytes
               3 Dir(s)  474,553,004,032 bytes free


In [19]:
from torch.nn import CrossEntropyLoss

In [20]:
for epoch in range(num_epochs):
    for batch in dataloader:
        # Mover los tensores al dispositivo adecuado
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        decoder_input_ids = batch["decoder_input_ids"].to(device)
        decoder_attention_mask = batch["decoder_attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Pasar el texto por el codificador BERT
        with torch.no_grad():
            encoder_outputs = encoder(input_ids=input_ids, attention_mask=attention_mask)
            
        # Pasar las salidas del codificador y las entradas del decodificador por el decodificador T5
        decoder_outputs = decoder(input_ids=decoder_input_ids,
                              attention_mask=decoder_attention_mask,
                              encoder_outputs=encoder_outputs.last_hidden_state,
                              labels=labels)

        # Calcular la pérdida y actualizar los pesos
        criterion = CrossEntropyLoss()
        logits = decoder_outputs.logits
        loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Imprimir la pérdida en el lote actual
        print(f"Loss: {loss.item()}")

    print(f"Epoch {epoch + 1}/{num_epochs} completed")

# Guardar los modelos entrenados
encoder.save_pretrained("encoder_trained")
decoder.save_pretrained("decoder_trained")


ValueError: not enough values to unpack (expected 3, got 2)

In [22]:
!pip install --upgrade transformers

     ---------------------------------------- 7.0/7.0 MB 40.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.24.0
    Uninstalling transformers-4.24.0:
      Successfully uninstalled transformers-4.24.0


In [ ]:
import torch
from transformers import BartForConditionalGeneration, BertModel

# Cargar el modelo pre-entrenado BART y el modelo de encoder de Bert-multilingual
bart_model = BartForConditionalGeneration.from_pretrained("philschmid/bart-large-cnn-samsum")
bert_model = BertModel.from_pretrained("bert-base-multilingual-cased")

# Extraer el decoder del modelo pre-entrenado BART
bart_decoder = bart_model.model.decoder

# Reemplazar el encoder del modelo pre-entrenado BART con el modelo de encoder de Bert-multilingual
bart_model.model.encoder = bert_model

# Ajustar el modelo pre-entrenado BART para que use el nuevo encoder
bart_model.eval()

# Ejemplo de cómo usar el modelo para generar resúmenes de texto
input_text = "Este es un ejemplo de texto que queremos resumir"
input_ids = bert_model.tokenizer.encode(input_text, add_special_tokens=True, return_tensors="pt")
summary_ids = bart_model.generate(input_ids)
summary = bart_model.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Texto de entrada: ", input_text)
print("Resumen generado: ", summary)



Downloading:  69%|████████████████████████████████████████████▍                   | 1.13G/1.63G [04:30<00:09, 52.1MB/s]

In [1]:
from transformers import BertTokenizer, BertModel, EncoderDecoderModel, AutoModelForSeq2SeqLM


C:\Users\josep\anaconda3\envs\nlp-39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_multilingual = BertModel.from_pretrained('bert-base-multilingual-cased')


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
model = EncoderDecoderModel(encoder=bert_multilingual, decoder=AutoModelForSeq2SeqLM.from_pretrained('t5-small'))


Downloading: 100%|█████████████████████████████████████████████████████████████████| 1.21k/1.21k [00:00<00:00, 805kB/s]
C:\Users\josep\anaconda3\envs\nlp-39\lib\site-packages\huggingface_hub\file_download.py:125: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\josep\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%

In [4]:
def summarize(text, model, tokenizer, max_length=150):
    inputs = tokenizer(text, return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=max_length, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [6]:
text = """Generative Pre-trained Transformer 3 (Transformador preentrenado generativo) , conocida por sus siglas (GPT-3), es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es la tercera generación de los modelos de predicción de lenguaje perteneciente a la serie GPT, creados por OpenAI, un laboratorio de investigación de inteligencia artificial con sede en San Francisco.1​ La versión completa de GPT-3 tiene una capacidad de 175.000 millones de parámetros de aprendizaje automatizado, lo cual supera la magnitud de su predecesor, GPT-2. GPT-3 fue introducido en mayo de 2020 y, hasta julio de 2020, se encontraba en fase beta.2​ Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas".3​ Previo a la liberación de GPT-3, el modelo de lenguaje más grande era Turing NLG desarrollado por Microsoft, presentado en febrero de 2020, con una capacidad diez veces menor que el de GPT-3."""
summary = summarize(text, model, tokenizer)
print("Resumen:", summary)


KeyError: 'input_ids'

In [10]:
from transformers import BertTokenizer, BertModel, EncoderDecoderModel, BertConfig

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_multilingual = BertModel.from_pretrained('bert-base-multilingual-cased')
decoder_config = BertConfig.from_pretrained('bert-base-multilingual-cased', is_decoder=True, add_cross_attention=True)

decoder_config.decoder_start_token_id = tokenizer.cls_token_id

bert_multilingual_decoder = BertModel.from_pretrained('bert-base-multilingual-cased', config=decoder_config)
model = EncoderDecoderModel(encoder=bert_multilingual, decoder=bert_multilingual_decoder)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'c

In [11]:
def summarize(text, model, tokenizer, max_length=150):
    inputs = tokenizer(text, return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=max_length, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [12]:
text = """Generative Pre-trained Transformer 3 (Transformador preentrenado generativo) , conocida por sus siglas (GPT-3), es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es la tercera generación de los modelos de predicción de lenguaje perteneciente a la serie GPT, creados por OpenAI, un laboratorio de investigación de inteligencia artificial con sede en San Francisco.1​ La versión completa de GPT-3 tiene una capacidad de 175.000 millones de parámetros de aprendizaje automatizado, lo cual supera la magnitud de su predecesor, GPT-2. GPT-3 fue introducido en mayo de 2020 y, hasta julio de 2020, se encontraba en fase beta.2​ Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas".3​ Previo a la liberación de GPT-3, el modelo de lenguaje más grande era Turing NLG desarrollado por Microsoft, presentado en febrero de 2020, con una capacidad diez veces menor que el de GPT-3."""
summary = summarize(text, model, tokenizer)
print("Resumen:", summary)


NotImplementedError: A model class needs to define a `prepare_inputs_for_generation` method in order to use `generate`.

In [30]:
import torch
from transformers import BertModel, BertTokenizer, T5ForConditionalGeneration, T5Tokenizer, EncoderDecoderModel

encoder = BertModel.from_pretrained('bert-base-multilingual-cased')
encoder_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

decoder = T5ForConditionalGeneration.from_pretrained('t5-small')
decoder_tokenizer = T5Tokenizer.from_pretrained('t5-small')


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
#from transformers import T5ForConditionalGeneration

#class CustomT5ForConditionalGeneration(T5ForConditionalGeneration):
#    def prepare_inputs_for_generation(self, input_ids, past=None, attention_mask=None, **kwargs):
#        return {"input_ids": input_ids, "attention_mask": attention_mask, "past_key_values": past}

#decoder = CustomT5ForConditionalGeneration.from_pretrained('t5-small')


In [31]:
model = EncoderDecoderModel(encoder=encoder, decoder=decoder)


In [32]:
#def summarize(text, model, encoder_tokenizer, decoder_tokenizer, max_length=150):
#    inputs = encoder_tokenizer(text, return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')
#    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=max_length, early_stopping=True)
#    summary = decoder_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#    return summary

#def summarize(text, model, encoder_tokenizer, decoder_tokenizer, max_length=150):
#    inputs = encoder_tokenizer(text, return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')
#    decoder_input_ids = torch.tensor([[decoder_tokenizer.pad_token_id]], dtype=torch.long)  # Add a '<pad>' token at the beginning
#    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=max_length, early_stopping=True, decoder_input_ids=decoder_input_ids)
#    summary = decoder_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#    return summary

def summarize(text, model, encoder_tokenizer, decoder_tokenizer, max_length=150):
    inputs = encoder_tokenizer(text, return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')
    encoder_outputs = model.encoder(inputs['input_ids'], attention_mask=inputs['attention_mask'], return_dict=True)
    decoder_input_ids = torch.tensor([[decoder_tokenizer.pad_token_id]], dtype=torch.long)  # Add a '<pad>' token at the beginning
    summary_ids = model.decoder.generate(decoder_input_ids, num_beams=4, max_length=max_length, early_stopping=True, encoder_outputs=encoder_outputs)
    summary = decoder_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary




In [33]:
text = """Generative Pre-trained Transformer 3 (Transformador preentrenado generativo) , conocida por sus siglas (GPT-3), es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es la tercera generación de los modelos de predicción de lenguaje perteneciente a la serie GPT, creados por OpenAI, un laboratorio de investigación de inteligencia artificial con sede en San Francisco.1​ La versión completa de GPT-3 tiene una capacidad de 175.000 millones de parámetros de aprendizaje automatizado, lo cual supera la magnitud de su predecesor, GPT-2. GPT-3 fue introducido en mayo de 2020 y, hasta julio de 2020, se encontraba en fase beta.2​ Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas".3​ Previo a la liberación de GPT-3, el modelo de lenguaje más grande era Turing NLG desarrollado por Microsoft, presentado en febrero de 2020, con una capacidad diez veces menor que el de GPT-3."""
summary = summarize(text, model, encoder_tokenizer, decoder_tokenizer)
print("Resumen:", summary)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (600x768 and 512x512)

In [2]:
# Instalación (si aún no lo has hecho)
# !pip install transformers

import torch
from transformers import BertTokenizer, EncoderDecoderModel

# Carga los modelos preentrenados
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')

# Agrega una capa de generación lineal a BERT
bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id
bert2bert.config.vocab_size = bert2bert.config.encoder.vocab_size

bert2bert.decoder.cls_token_id = bert2bert.decoder.bert.embeddings.word_embeddings.weight.new_zeros(1).long().item()
bert2bert.decoder.sep_token_id = bert2bert.decoder.bert.embeddings.word_embeddings.weight.new_ones(1).long().item()
bert2bert.decoder.pad_token_id = bert2bert.decoder.bert.embeddings.word_embeddings.weight.new_full((1,), 2).long().item()


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.wei

In [3]:
# Un ejemplo de entrada de texto para resumir
input_text = """Generative Pre-trained Transformer 3 (Transformador preentrenado generativo) , conocida por sus siglas (GPT-3), es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es la tercera generación de los modelos de predicción de lenguaje perteneciente a la serie GPT, creados por OpenAI, un laboratorio de investigación de inteligencia artificial con sede en San Francisco.1​ La versión completa de GPT-3 tiene una capacidad de 175.000 millones de parámetros de aprendizaje automatizado, lo cual supera la magnitud de su predecesor, GPT-2. GPT-3 fue introducido en mayo de 2020 y, hasta julio de 2020, se encontraba en fase beta.2​ Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas".3​ Previo a la liberación de GPT-3, el modelo de lenguaje más grande era Turing NLG desarrollado por Microsoft, presentado en febrero de 2020, con una capacidad diez veces menor que el de GPT-3."""
input_tokens = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Genera el resumen
summary_ids = bert2bert.generate(input_tokens["input_ids"], num_beams=5, max_length=30, early_stopping=True)
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Resumen generado:", summary_text)

C:\Users\josep\anaconda3\envs\nlp-39\lib\site-packages\transformers\generation\utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Resumen generado: ##litlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlitlit


In [5]:
import pandas as pd

# Cargar el DataFrame
data = pd.DataFrame({ "texto" : ["""Generative Pre-trained Transformer 3 (Transformador preentrenado generativo) , conocida por sus siglas (GPT-3), es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es la tercera generación de los modelos de predicción de lenguaje perteneciente a la serie GPT, creados por OpenAI, un laboratorio de investigación de inteligencia artificial con sede en San Francisco.1​ La versión completa de GPT-3 tiene una capacidad de 175.000 millones de parámetros de aprendizaje automatizado, lo cual supera la magnitud de su predecesor, GPT-2. GPT-3 fue introducido en mayo de 2020 y, hasta julio de 2020, se encontraba en fase beta.2​ Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas".3​ Previo a la liberación de GPT-3, el modelo de lenguaje más grande era Turing NLG desarrollado por Microsoft, presentado en febrero de 2020, con una capacidad diez veces menor que el de GPT-3."""]*100,
                       "resumen_texto" : ["""GPT-3 es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas"""]*100})
data

,texto,resumen_texto
0,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
1,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
2,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
3,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
4,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
...,...,...
95,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
96,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
97,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...
98,Generative Pre-trained Transformer 3 (Transfor...,GPT-3 es un modelo de lenguaje autorregresivo ...


In [1]:
import torch
import pandas as pd
from transformers import BertTokenizer, EncoderDecoderModel, Seq2SeqTrainingArguments, Seq2SeqTrainer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Clase personalizada para manejar tu conjunto de datos
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.iloc[index]['texto']
        summary = self.data.iloc[index]['resumen_texto']

        input_tokens = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        target_tokens = self.tokenizer(summary, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")

        return {
            'input_ids': input_tokens['input_ids'].squeeze(),
            'attention_mask': input_tokens['attention_mask'].squeeze(),
            'labels': target_tokens['input_ids'].squeeze(),
        }

# Cambia esto con la ubicación de tu dataframe
df = pd.DataFrame({ "texto" : ["""Generative Pre-trained Transformer 3 (Transformador preentrenado generativo) , conocida por sus siglas (GPT-3), es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es la tercera generación de los modelos de predicción de lenguaje perteneciente a la serie GPT, creados por OpenAI, un laboratorio de investigación de inteligencia artificial con sede en San Francisco.1​ La versión completa de GPT-3 tiene una capacidad de 175.000 millones de parámetros de aprendizaje automatizado, lo cual supera la magnitud de su predecesor, GPT-2. GPT-3 fue introducido en mayo de 2020 y, hasta julio de 2020, se encontraba en fase beta.2​ Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas".3​ Previo a la liberación de GPT-3, el modelo de lenguaje más grande era Turing NLG desarrollado por Microsoft, presentado en febrero de 2020, con una capacidad diez veces menor que el de GPT-3."""]*100,
                       "resumen_texto" : ["""GPT-3 es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas"""]*100})

train_df, val_df = train_test_split(df, test_size=0.1)

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')

## Configura el token de inicio del decodificador y el token de padding
#model.config.decoder_start_token_id = tokenizer.cls_token_id
#model.config.pad_token_id = tokenizer.pad_token_id

# Agrega una capa de generación lineal a BERT
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

model.decoder.cls_token_id = model.decoder.bert.embeddings.word_embeddings.weight.new_zeros(1).long().item()
model.decoder.sep_token_id = model.decoder.bert.embeddings.word_embeddings.weight.new_ones(1).long().item()
model.decoder.pad_token_id = model.decoder.bert.embeddings.word_embeddings.weight.new_full((1,), 2).long().item()

C:\Users\josep\anaconda3\envs\nlp-39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be

In [2]:
# Preparación de los datos
max_length = 128
batch_size = 4
n_train_epochs = 2

train_dataset = CustomDataset(train_df, tokenizer, max_length)
val_dataset = CustomDataset(val_df, tokenizer, max_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Configuración del entrenamiento
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=10,
    save_steps=500,
    save_total_limit=3,
    num_train_epochs=n_train_epochs,
    fp16=True,
    gradient_accumulation_steps=1,
    report_to="none",
)


In [3]:
# Entrenamiento
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

C:\Users\josep\anaconda3\envs\nlp-39\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\josep\anaconda3\envs\nlp-39\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss


TrainOutput(global_step=46, training_loss=3.952505628051965, metrics={'train_runtime': 117.5536, 'train_samples_per_second': 1.531, 'train_steps_per_second': 0.391, 'total_flos': 27617741337600.0, 'train_loss': 3.952505628051965, 'epoch': 2.0})

In [6]:
def generate_summary(text, model, tokenizer, max_input_length=512, max_output_length=128):
    # Tokenizar el texto de entrada
    input_tokens = tokenizer.encode(text, return_tensors="pt", max_length=max_input_length, truncation=True)

    # Mover los tokens a la GPU si está disponible
    input_tokens = input_tokens.to(model.device)

    # Generar resumen
    summary_ids = model.generate(input_tokens, num_return_sequences=1, max_length=max_output_length)

    # Decodificar y devolver el resumen
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [7]:
text = """Generative Pre-trained Transformer 3 (Transformador preentrenado generativo) , conocida por sus siglas (GPT-3), es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es la tercera generación de los modelos de predicción de lenguaje perteneciente a la serie GPT, creados por OpenAI, un laboratorio de investigación de inteligencia artificial con sede en San Francisco.1​ La versión completa de GPT-3 tiene una capacidad de 175.000 millones de parámetros de aprendizaje automatizado, lo cual supera la magnitud de su predecesor, GPT-2. GPT-3 fue introducido en mayo de 2020 y, hasta julio de 2020, se encontraba en fase beta.2​ Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas".3​ Previo a la liberación de GPT-3, el modelo de lenguaje más grande era Turing NLG desarrollado por Microsoft, presentado en febrero de 2020, con una capacidad diez veces menor que el de GPT-3."""
summary = generate_summary(text, model, tokenizer)
print("Resumen:", summary)

Resumen: GP GP GP GP GP GP GP GP GP GP GP GP GP GP GP GP GP GPT GP GP GP GP GP GP GP GP GP GP GP GP GP GPT - - - GP GP GP GP GPT - GP GP GP GP GP GP GP GPT - 3 es es un un un un un un modelo de autor un un modelo de lenguaje autor autorresivo que emplea un un modelo de lenguaje autorregresivo que emplea un un un modelo de lenguaje autor de lenguaje autor de lenguaje autorregresivo que emplea aprendizaje de lenguaje autorregresivo que emplea aprendizaje profundo que emplea aprendizaje profundo


In [5]:
# Un ejemplo de entrada de texto para resumir
input_text = """Generative Pre-trained Transformer 3 (Transformador preentrenado generativo) , conocida por sus siglas (GPT-3), es un modelo de lenguaje autorregresivo que emplea aprendizaje profundo para producir textos que simulan la redacción humana. Es la tercera generación de los modelos de predicción de lenguaje perteneciente a la serie GPT, creados por OpenAI, un laboratorio de investigación de inteligencia artificial con sede en San Francisco.1​ La versión completa de GPT-3 tiene una capacidad de 175.000 millones de parámetros de aprendizaje automatizado, lo cual supera la magnitud de su predecesor, GPT-2. GPT-3 fue introducido en mayo de 2020 y, hasta julio de 2020, se encontraba en fase beta.2​ Es parte de una tendencia en sistemas de procesamiento de lenguaje natural (NLP) basados en "representaciones de lenguaje pre-entrenadas".3​ Previo a la liberación de GPT-3, el modelo de lenguaje más grande era Turing NLG desarrollado por Microsoft, presentado en febrero de 2020, con una capacidad diez veces menor que el de GPT-3."""
input_tokens = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Genera el resumen
summary_ids = model.generate(input_tokens["input_ids"], num_beams=5, max_length=30, early_stopping=True)
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Resumen generado:", summary_text)

C:\Users\josep\anaconda3\envs\nlp-39\lib\site-packages\transformers\generation\utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)